In [1]:
def simple_hash(key, array_size):
    """
    一个简单的字符串哈希函数。
    :param key: 输入的键（字符串）
    :param array_size: 哈希表数组的大小
    :return: 计算得到的数组索引
    """
    total = 0
    for char in key:
        # 将字符转换为其 ASCII 码值并累加
        total += ord(char)
    # 对数组大小取模，确保索引在数组范围内
    return total % array_size

# 测试我们的哈希函数
keys_to_test = ["Alice", "Bob", "Charlie", "David"]
array_size = 10

print("简单哈希函数测试结果：")
for key in keys_to_test:
    hash_index = simple_hash(key, array_size)
    print(f"  键 '{key}' -> 哈希索引: {hash_index}")

简单哈希函数测试结果：
  键 'Alice' -> 哈希索引: 8
  键 'Bob' -> 哈希索引: 5
  键 'Charlie' -> 哈希索引: 6
  键 'David' -> 哈希索引: 8


In [1]:
class HashTableChaining:
    """使用链地址法解决冲突的哈希表实现。"""

    def __init__(self, size=10):
        """初始化哈希表。
        :param size: 底层数组的初始大小
        """
        self.size = size
        # 创建一个大小为 `size` 的列表，每个元素初始化为一个空列表（代表链表）
        self.table = [[] for _ in range(size)]

    def _hash(self, key):
        """内部哈希函数。这里使用Python内置的hash函数并取模。"""
        return hash(key) % self.size

    def insert(self, key, value):
        """向哈希表中插入一个键值对。"""
        index = self._hash(key)
        bucket = self.table[index] # 获取该索引对应的"桶"（链表）

        # 遍历桶，检查键是否已存在
        for i, (k, v) in enumerate(bucket):
            if k == key:
                # 键已存在，更新值
                bucket[i] = (key, value)
                return
        # 键不存在，添加到链表末尾
        bucket.append((key, value))

    def get(self, key):
        """根据键获取值。如果键不存在则返回None。"""
        index = self._hash(key)
        bucket = self.table[index]

        for k, v in bucket:
            if k == key:
                return v
        return None # 键不存在

    def delete(self, key):
        """根据键删除键值对。"""
        index = self._hash(key)
        bucket = self.table[index]

        for i, (k, v) in enumerate(bucket):
            if k == key:
                del bucket[i] # 从链表中删除该节点
                return True # 删除成功
        return False # 键不存在，删除失败

    def display(self):
        """打印哈希表的所有内容。"""
        for i, bucket in enumerate(self.table):
            print(f"索引 {i}: {bucket}")

# 使用示例
print("\n--- 链地址法哈希表示例 ---")
phone_book = HashTableChaining(5)
phone_book.insert("Alice", "123-4567")
phone_book.insert("Bob", "987-6543")
phone_book.insert("Charlie", "555-1234")
# 假设"David"的哈希值与"Alice"冲突（为了演示）
phone_book.insert("David", "111-2222")

print("插入数据后的哈希表：")
phone_book.display()

print(f"\n查找 'Bob' 的电话: {phone_book.get('Bob')}")
print(f"查找不存在的 'Eve': {phone_book.get('Eve')}")

phone_book.delete("Charlie")
print("\n删除 'Charlie' 后的哈希表：")
phone_book.display()


--- 链地址法哈希表示例 ---
插入数据后的哈希表：
索引 0: [('Charlie', '555-1234')]
索引 1: []
索引 2: [('David', '111-2222')]
索引 3: [('Alice', '123-4567')]
索引 4: [('Bob', '987-6543')]

查找 'Bob' 的电话: 987-6543
查找不存在的 'Eve': None

删除 'Charlie' 后的哈希表：
索引 0: []
索引 1: []
索引 2: [('David', '111-2222')]
索引 3: [('Alice', '123-4567')]
索引 4: [('Bob', '987-6543')]


In [2]:
class HashTableLinearProbing:
    """使用线性探测开放地址法解决冲突的哈希表实现。"""

    def __init__(self, size=10):
        self.size = size
        # 使用一个特殊标记 `None` 表示空位，使用一个标记（如 `'DELETED'`）表示已删除的位置
        # 为了简化，这里只用 `None` 表示空位，查找时遇到 `None` 就停止
        self.keys = [None] * size
        self.values = [None] * size

    def _hash(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        index = self._hash(key)
        original_index = index

        # 线性探测寻找空位或相同的键
        while self.keys[index] is not None:
            if self.keys[index] == key:
                # 键已存在，更新值
                self.values[index] = value
                return
            index = (index + 1) % self.size # 移动到下一个位置（循环数组）
            if index == original_index:
                # 已经找了一圈，表满了（实际应用中应该扩容）
                raise Exception("哈希表已满")

        # 找到空位，插入
        self.keys[index] = key
        self.values[index] = value

    def get(self, key):
        index = self._hash(key)
        original_index = index

        while self.keys[index] is not None:
            if self.keys[index] == key:
                return self.values[index]
            index = (index + 1) % self.size
            if index == original_index:
                break # 找了一圈没找到
        return None

    def display(self):
        for i in range(self.size):
            if self.keys[i] is not None:
                print(f"索引 {i}: 键={self.keys[i]}, 值={self.values[i]}")
            else:
                print(f"索引 {i}: 空")

# 使用示例
print("\n--- 线性探测哈希表示例 ---")
ht_linear = HashTableLinearProbing(7) # 用小尺寸容易看到探测
ht_linear.insert("Apple", 10)
ht_linear.insert("Banana", 20)
ht_linear.insert("Cherry", 30)
# 假设"Date"和"Apple"哈希冲突
ht_linear.insert("Date", 40)

print("插入数据后的哈希表：")
ht_linear.display()

print(f"\n查找 'Banana': {ht_linear.get('Banana')}")
print(f"查找 'Date' (冲突后插入的): {ht_linear.get('Date')}")


--- 线性探测哈希表示例 ---
插入数据后的哈希表：
索引 0: 键=Cherry, 值=30
索引 1: 键=Date, 值=40
索引 2: 空
索引 3: 空
索引 4: 键=Apple, 值=10
索引 5: 键=Banana, 值=20
索引 6: 空

查找 'Banana': 20
查找 'Date' (冲突后插入的): 40


In [3]:
# 练习：使用我们实现的 HashTableChaining 来统计词频
print("\n=== 实践练习：单词计数器 ===")

# 1. 创建哈希表实例
word_counter = HashTableChaining(size=10)

# 2. 提供的测试文本
text = "the quick brown fox jumps over the lazy dog the dog is not lazy at all"
words = text.split() # 将文本分割成单词列表

print("处理的单词列表:", words)

# 3. 遍历单词，插入哈希表
for word in words:
    current_count = word_counter.get(word)
    if current_count is None:
        # 单词第一次出现，计数为1
        word_counter.insert(word, 1)
    else:
        # 单词已存在，计数加1
        word_counter.insert(word, current_count + 1)

# 4. 输出统计结果
print("\n单词出现次数统计：")
# 注意：我们的display方法会按索引打印，这里我们写一个更友好的输出
all_entries = []
for bucket in word_counter.table:
    all_entries.extend(bucket) # 将所有桶中的条目合并

for word, count in all_entries:
    print(f"  '{word}': {count} 次")

# 5. 验证特定单词
test_word = "the"
print(f"\n验证：单词 '{test_word}' 出现了 {word_counter.get(test_word)} 次。")
test_word = "lazy"
print(f"验证：单词 '{test_word}' 出现了 {word_counter.get(test_word)} 次。")


=== 实践练习：单词计数器 ===
处理的单词列表: ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', 'the', 'dog', 'is', 'not', 'lazy', 'at', 'all']

单词出现次数统计：
  'fox': 1 次
  'jumps': 1 次
  'over': 1 次
  'not': 1 次
  'the': 3 次
  'is': 1 次
  'lazy': 2 次
  'dog': 2 次
  'all': 1 次
  'at': 1 次
  'brown': 1 次
  'quick': 1 次

验证：单词 'the' 出现了 3 次。
验证：单词 'lazy' 出现了 2 次。
